---
title: "Matter as Topological Defects: Vortex Stability"
author: "Raúl Chiclano"
date: "2025-11-30"
categories: [matter, topology]
description: "Demonstrating the stability of solitons in the superfluid vacuum."
format:
  html:
    code-fold: true
execute:
  freeze: true 
---

We simulate the Gross-Pitaevskii Equation in 2D using the Split-Step Fourier Method. We introduce a phase singularity (winding number Q=1) and relax the system in imaginary time to find the ground state.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML  # <--- NECESARIO PARA WEB

# --- 1. CONFIGURACIÓN ---
N = 128
L = 30.0
dt = 0.005
dt_im = 0.005
g = 2.0
rho_0 = 1.0

x = np.linspace(-L/2, L/2, N)
y = np.linspace(-L/2, L/2, N)
X, Y = np.meshgrid(x, y)
dx = x[1] - x[0]
k = 2 * np.pi * np.fft.fftfreq(N, d=dx)
KX, KY = np.meshgrid(k, k)
K2 = KX**2 + KY**2

# Potencial de trampa suave
R_grid = np.sqrt(X**2 + Y**2)
V_trap = 0.5 * (R_grid / (0.45*L))**10
V_trap[V_trap > 100] = 100

# --- 2. CONDICIÓN INICIAL ---
Theta = np.arctan2(Y, X)
xi = 1.0 / np.sqrt(g * rho_0)
Psi = np.sqrt(rho_0) * np.tanh(R_grid / (2.0 * xi)) * np.exp(1j * Theta)

# --- 3. RELAJACIÓN (TIEMPO IMAGINARIO) ---
# (Reducimos pasos para que la compilación web no tarde una eternidad)
U_kin_im = np.exp(-(K2 / 2.0) * dt_im)

for i in range(100): 
    density = np.abs(Psi)**2
    Psi = Psi * np.exp(-(V_trap + g * density) * (dt_im / 2))
    Psi_k = np.fft.fft2(Psi)
    Psi_k *= U_kin_im
    Psi = np.fft.ifft2(Psi_k)
    density = np.abs(Psi)**2
    Psi = Psi * np.exp(-(V_trap + g * density) * (dt_im / 2))
    norm_factor = np.sqrt(rho_0) / np.max(np.abs(Psi))
    Psi *= norm_factor
    Psi = np.abs(Psi) * np.exp(1j * Theta)

# --- 4. EVOLUCIÓN REAL Y ANIMACIÓN ---
U_kin_real = np.exp(-1j * (K2 / 2.0) * dt)

def evolution_step_real(psi_in):
    psi_mod = psi_in * np.exp(-1j * (V_trap + g * np.abs(psi_in)**2) * (dt / 2))
    psi_k = np.fft.fft2(psi_mod)
    psi_k *= U_kin_real
    psi_mod = np.fft.ifft2(psi_k)
    psi_out = psi_mod * np.exp(-1j * (V_trap + g * np.abs(psi_mod)**2) * (dt / 2))
    return psi_out

# Configuración de la figura para la web
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
plt.close() # <--- IMPORTANTE: Evita que se muestre una imagen estática vacía antes de la animación

im1 = ax1.imshow(np.abs(Psi)**2, extent=[-L/2, L/2, -L/2, L/2], cmap='inferno')
ax1.set_title("Density")
im2 = ax2.imshow(np.angle(Psi), extent=[-L/2, L/2, -L/2, L/2], cmap='hsv')
ax2.set_title("Phase")

def update(frame):
    global Psi
    for _ in range(5):
        Psi = evolution_step_real(Psi)
    im1.set_data(np.abs(Psi)**2)
    im2.set_data(np.angle(Psi))
    return im1, im2

# Generar la animación
ani = animation.FuncAnimation(fig, update, frames=60, interval=50, blit=True)

# RENDERIZAR COMO VIDEO HTML5 (Base64 incrustado)
# Esto es portable (un solo archivo HTML) y más ligero que JSHTML
from IPython.display import HTML
display(HTML(ani.to_html5_video()))
plt.close()